![alt text](./Cerny_logo_1.jpg)

# Analysis of Cerny ventilation recordings

Analysis of ventilator data obtained from ELBW (birth weight <1000 g) infants transferred during the first 24 hours of life among recordings `AL000001 - AL001100` and  `AT000001 - AT000818`

#### Author: Dr Gusztav Belteki

### 1. Import the required libraries and set options

In [ ]:
import IPython
import pandas as pd
import numpy as np
import scipy as sp
import matplotlib
import matplotlib.pyplot as plt

import os
import sys
import pickle

from scipy import stats
from pandas import Series, DataFrame
from datetime import datetime, timedelta

%matplotlib inline
matplotlib.style.use('classic')
matplotlib.rcParams['figure.facecolor'] = 'w'

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('mode.chained_assignment', None) 

In [ ]:
print("Python version: {}".format(sys.version))
print("pandas version: {}".format(pd.__version__))
print("matplotlib version: {}".format(matplotlib.__version__))
print("NumPy version: {}".format(np.__version__))
print("SciPy version: {}".format(sp.__version__))
print("IPython version: {}".format(IPython.__version__))

### 2. List and set the working directory and the directory to write out data

In [ ]:
# Topic of the Notebook which will also be the name of the subfolder containing results
TOPIC = 'ELBW'

# Path to project folder containing clinical data (current weights only) and for export of results
PATH = os.path.join(os.sep, 'Users', 'guszti', 'Library', 'Mobile Documents', 'com~apple~CloudDocs', 
                            'Documents', 'Research', 'Ventilation')

# Name of the external hard drive
DRIVE = 'GUSZTI'

# Data loaded from both directories
DIR_READ_1 = os.path.join(os.sep, 'Volumes', DRIVE, 'data_dump', 'fabian')
DIR_READ_2 = os.path.join(os.sep, 'Volumes', DRIVE, 'data_dump', 'fabian_new')

# Results will be written in this folder
DIR_WRITE =  os.path.join(os.sep, PATH, 'ventilation_fabian_combined', 'Analyses', TOPIC)
os.makedirs(DIR_WRITE, exist_ok=True)

# Images and raw data will be written on an external hard drive
DATA_DUMP = os.path.join(os.sep, 'Volumes', DRIVE, 'data_dump', 'fabian_combined', TOPIC)
os.makedirs(DATA_DUMP, exist_ok=True)

In [ ]:
DIR_READ_1, DIR_READ_2, DIR_WRITE, DATA_DUMP

### 3. Import clinical data, ventilation modes and blood gases of the final dataset

In [ ]:
# These data were exported by the `ELBW_analysis_clinical` Notebook

# Computationally collected data
with open(os.path.join(DATA_DUMP, 'clin_data_comput_ELBW.pickle'), 'rb') as handle:
    clin_data_comput_ELBW = pickle.load(handle)

with open(os.path.join(DATA_DUMP, 'ventilation_modes_ELBW.pickle'), 'rb') as handle:
    vent_modes_ELBW = pickle.load(handle)

with open(os.path.join(DATA_DUMP, 'blood_gases_comput.pickle'), 'rb') as handle:
    blood_gases_comput = pickle.load(handle)

# Manually collected data
# Demographic, clinical and outcome data
with open(os.path.join(DATA_DUMP, 'clin_data_combined.pickle'), 'rb') as handle:
    clinical_data_combined = pickle.load(handle)

# Vital parameters for the final dataset
with open(os.path.join(DATA_DUMP, 'vital_parameters_manual.pickle'), 'rb') as handle:
    vital_parameters = pickle.load(handle)
    
# Blood gases for the final dataset
with open(os.path.join(DATA_DUMP, 'blood_gases_manual.pickle'), 'rb') as handle:
    blood_gases_manual = pickle.load(handle)   

### 4. Import ventilator data from pickle archives

#### A. Import data on ventilated recordings

In [ ]:
with open(os.path.join(DIR_READ_1, 'data_pars_measurements_ventilated_1_1100.pickle'), 'rb') as handle:
    data_pars_measurements_1_1100 = pickle.load(handle)
with open(os.path.join(DIR_READ_1, 'data_pars_settings_ventilated_1_1100.pickle'), 'rb') as handle:
    data_pars_settings_1_1100 = pickle.load(handle)
   
with open(os.path.join(DIR_READ_2, 'data_pars_measurements_ventilated_new_1_1305.pickle'), 'rb') as handle:
    data_pars_measurements_new_1_1305 = pickle.load(handle)
with open(os.path.join(DIR_READ_2, 'data_pars_settings_ventilated_new_1_1305.pickle'), 'rb') as handle:
    data_pars_settings_new_1_1305 = pickle.load(handle)

data_pars_measurements_ventilated = {**data_pars_measurements_1_1100, **data_pars_measurements_new_1_1305}
data_pars_settings_ventilated =     {**data_pars_settings_1_1100, **data_pars_settings_new_1_1305}

len(data_pars_measurements_ventilated), len(data_pars_settings_ventilated)

#### B. Import data on noninvasive recordings

In [ ]:
with open(os.path.join(DIR_READ_1, 'data_pars_measurements_noninvasive_1_300.pickle'), 'rb') as handle:
    data_pars_measurements_1_300 = pickle.load(handle)
with open(os.path.join(DIR_READ_1, 'data_pars_settings_noninvasive_1_300.pickle'), 'rb') as handle:
    data_pars_settings_1_300 = pickle.load(handle)

with open(os.path.join(DIR_READ_1, 'data_pars_measurements_noninvasive_301_600.pickle'), 'rb') as handle:
    data_pars_measurements_301_600 = pickle.load(handle)
with open(os.path.join(DIR_READ_1, 'data_pars_settings_noninvasive_301_600.pickle'), 'rb') as handle:
    data_pars_settings_301_600 = pickle.load(handle)

with open(os.path.join(DIR_READ_1, 'data_pars_measurements_noninvasive_601_900.pickle'), 'rb') as handle:
    data_pars_measurements_601_900 = pickle.load(handle)
with open(os.path.join(DIR_READ_1, 'data_pars_settings_noninvasive_601_900.pickle'), 'rb') as handle:
    data_pars_settings_601_900 = pickle.load(handle)

with open(os.path.join(DIR_READ_1, 'data_pars_measurements_noninvasive_901_1100.pickle'), 'rb') as handle:
    data_pars_measurements_901_1100 = pickle.load(handle)
with open(os.path.join(DIR_READ_1, 'data_pars_settings_noninvasive_901_1100.pickle'), 'rb') as handle:
    data_pars_settings_901_1100 = pickle.load(handle)
   
with open(os.path.join(DIR_READ_2, 'data_pars_measurements_noninvasive_new_1_1305.pickle'), 'rb') as handle:
    data_pars_measurements_new_1_1305 = pickle.load(handle)
with open(os.path.join(DIR_READ_2, 'data_pars_settings_noninvasive_new_1_1305.pickle'), 'rb') as handle:
    data_pars_settings_new_1_1305 = pickle.load(handle)

data_pars_measurements_noninvasive = {**data_pars_measurements_1_300, **data_pars_measurements_301_600, 
                                      **data_pars_measurements_601_900, **data_pars_measurements_901_1100,
                                      **data_pars_measurements_new_1_1305}
data_pars_settings_noninvasive =     {**data_pars_settings_1_300, **data_pars_settings_301_600,
                                      **data_pars_settings_601_900, **data_pars_settings_901_1100,
                                      **data_pars_settings_new_1_1305}

len(data_pars_measurements_noninvasive), len(data_pars_settings_noninvasive)

#### C. Limit ventilator data to the included cases

In [ ]:
data_pars_measurements_ventilated = {rec: data_pars_measurements_ventilated[rec] 
                for rec in data_pars_measurements_ventilated if rec in clin_data_comput_ELBW.index}
data_pars_settings_ventilated = {rec: data_pars_settings_ventilated[rec] 
                for rec in data_pars_settings_ventilated if rec in clin_data_comput_ELBW.index}

len(data_pars_measurements_ventilated), len(data_pars_settings_ventilated)

In [ ]:
data_pars_measurements_noninvasive = {rec: data_pars_measurements_noninvasive[rec] 
                for rec in data_pars_measurements_noninvasive if rec in clin_data_comput_ELBW.index}
data_pars_settings_noninvasive = {rec: data_pars_settings_noninvasive[rec] 
                for rec in data_pars_settings_noninvasive if rec in clin_data_comput_ELBW.index}

len(data_pars_measurements_noninvasive), len(data_pars_settings_noninvasive)

In [ ]:
set(data_pars_measurements_ventilated.keys()) & set(data_pars_measurements_noninvasive.keys())

In [ ]:
data_pars_measurements = {**data_pars_measurements_ventilated, **data_pars_measurements_noninvasive}
data_pars_settings = {**data_pars_settings_ventilated, **data_pars_settings_noninvasive}
len(data_pars_measurements), len(data_pars_settings)

### 5. Keep only selected parameters and correct data types when needed

#### A. Keep only parameters and settings relevant for the study

In [ ]:
parameters_to_choose_from = {'PIP', 'MAP', 'PEEP',  'MVresp', 'Leak', 'RR', 'FiO2', 'Flow_exp', 'MV_kg', 
                             'VTimand_kg', 'VTemand_kg', 'VTespon_pat_kg', 'VTemand_resp_kg', }

settings_to_choose_from = {'Ventilator_mode', 'PIP_set', 'PEEP_set', 'FiO2_set', 'Flow_insp_set', 'Flow_exp_set', 
                           'Ti_set', 'Te_set', 'RR_set', 'IE_I_set', 'IE_E_set', 'VG_set_kg', 'VG_state', }

for recording, dta in data_pars_measurements.items():
    parameters_to_keep = sorted(set(dta.columns) & parameters_to_choose_from)
    data_pars_measurements[recording] = dta[parameters_to_keep]
    
for recording, dta in data_pars_settings.items():
    settings_to_keep = sorted(set(dta.columns) & settings_to_choose_from)
    data_pars_settings[recording] = dta[settings_to_keep]

#### B. Correct data types

In [ ]:
# All measured parameters are floats
for patient in data_pars_measurements:
    data_pars_measurements[patient] = data_pars_measurements[patient].astype('float')

In [ ]:
# Settings are floats or categorical

categorical_settings = ['Ventilator_mode',  'VG_state',]

float_settings = ['PIP_set', 'PEEP_set', 'FiO2_set', 'Flow_insp_set', 'Flow_exp_set', 
                  'Ti_set', 'Te_set', 'RR_set', 'IE_I_set', 'IE_E_set', 'VG_set_kg',]

for recording in data_pars_settings:
    for par in float_settings:
        if par in data_pars_settings[recording]:
            data_pars_settings[recording][par] = data_pars_settings[recording][par].astype('float')
    for par in categorical_settings:
        if par in data_pars_settings[recording]:
            data_pars_settings[recording][par] = data_pars_settings[recording][par].astype('category')  

#### C. Combine parameters and settings

In [ ]:
data_all = {}

for case in data_pars_measurements.keys():
    vent_pars = data_pars_measurements[case]
    vent_settings = data_pars_settings[case]
    data_all[case] = pd.concat([vent_pars, vent_settings], axis=1)

### 6. Cleanup ventilator data by removing outliers due to artifacts

In [ ]:
# Identify and remove abnormal inflations (presumaby ventilation artifacts)

data_points_before_cleanup = {}
for patient in data_all:
    data_points_before_cleanup[patient] = len(data_all[patient])
data_points_before_cleanup;

#### A. Inflations with negative inflating pressure
These are artifacts

In [ ]:
Pinfl_negative = {}
for patient in data_all:
    if 'PIP'in data_all[patient].columns and 'PEEP' in data_all[patient].columns:
        Pinfl_negative[patient] = sum(data_all[patient]['PIP'] < data_all[patient]['PEEP'])
Pinfl_negative;

In [ ]:
# How many occurrence in the whole dataset
sum(n for n in Pinfl_negative.values())

#### B. Inflations with VTemand_kg, VTemand_resp_kg or VTespon_respo_kg >25 mL/kg

Expired VT > 25 mL/kg is likely artifact, representing a flow sensor problem or an open circuit.

In [ ]:
VTemand_kg_over25 = {}
for patient in data_all:
    if 'VTemand_kg' in data_all[patient].columns:
        VTemand_kg_over25[patient] = (sum(data_all[patient]['VTemand_kg'] > 25))
VTemand_kg_over25;

In [ ]:
# How many occurrence in the whole dataset
sum(n for n in VTemand_kg_over25.values())

In [ ]:
VTemand_resp_kg_over25 = {}
for patient in data_all:
    if 'VTemand_resp_kg' in data_all[patient].columns:
        VTemand_resp_kg_over25[patient] = (sum(data_all[patient]['VTemand_resp_kg'] > 25))
VTemand_resp_kg_over25;

In [ ]:
# How many occurrence in the whole dataset
sum(n for n in VTemand_resp_kg_over25.values())

In [ ]:
VTespon_pat_kg_over25 = {}
for patient in data_all:
    if 'VTespon_pat_kg' in data_all[patient].columns:
        VTespon_pat_kg_over25[patient] = (sum(data_all[patient]['VTespon_pat_kg'] > 25))
VTespon_pat_kg_over25;

In [ ]:
# How many occurrence in the whole dataset
sum(n for n in VTespon_pat_kg_over25.values())

#### C. Inflations with MV over 1 L/min/kg

This will not occur during conventional ventilation, unless due to an artifact (eg circuit is open). 

In [ ]:
MV_over_1 = {}
for patient in data_all:
    if 'MV_kg' in data_all[patient].columns:
        MV_over_1[patient] = sum(data_all[patient]['MV_kg'] > 1) 
MV_over_1;

In [ ]:
# How many occurrence in the whole dataset
sum(n for n in MV_over_1.values())

#### D. Inflations with RR  > 130/min

These values are likely abnormal representing autocycling due to condensed water in the circuit

In [ ]:
RR_over_130 = {}
for patient in data_all:
    if 'RR' in data_all[patient].columns:
        RR_over_130[patient] = sum(data_all[patient]['RR'] > 130)
RR_over_130;

In [ ]:
# How many occurrence in the whole dataset
sum(n for n in RR_over_130.values())

#### E. Remove these anomalies from the dataset

Keep rows with missing data of the parameters because other parameters at the same time point may be present and valid

In [ ]:
%%time

for patient in data_all:
    #print(patient)
    dta = data_all[patient]
    if 'PIP' in dta and 'PEEP' in dta:
        dta = dta[dta['PIP'].isna() | dta['PEEP'].isna() | (dta['PIP'] >= dta['PEEP'])]
    if 'VTemand_kg' in dta:
        dta = dta[dta['VTemand_kg'].isna() | (dta['VTemand_kg'] <= 25)]
    if 'VTemand_resp_kg' in dta:
        dta = dta[dta['VTemand_resp_kg'].isna() | (dta['VTemand_resp_kg'] <= 25)]
    if 'VTespon_pat_kg' in dta:
        dta = dta[dta['VTespon_pat_kg'].isna() | (dta['VTespon_pat_kg'] <= 25)]
    if 'MV_kg' in dta:
        dta = dta[dta['MV_kg'].isna() | (dta['MV_kg'] <= 1)]
    if 'RR' in dta:
        dta = dta[dta['RR'].isna() | (dta['RR'] <= 130)]
    data_all[patient] = dta

In [ ]:
data_points_after_cleanup = {}
for patient in data_all:
    data_points_after_cleanup[patient] = len(data_all[patient])
data_points_after_cleanup;

In [ ]:
cleanup_frme = DataFrame([data_points_before_cleanup, data_points_after_cleanup, Pinfl_negative, 
    VTemand_kg_over25, VTemand_resp_kg_over25, VTespon_pat_kg_over25, MV_over_1, RR_over_130]).T
cleanup_frme.columns =   ['data_points_before_cleanup', 'data_points_after_cleanup', 'Pinfl_negative', 
    'VTemand_kg_over25', 'VTemand_resp_kg_over25', 'VTespon_pat_kg_over25', 'MV_over_1', 'RR_over_130',]
cleanup_frme['rows_removed'] = cleanup_frme['data_points_before_cleanup'] - \
                                cleanup_frme['data_points_after_cleanup']
cleanup_frme.sum()

In [ ]:
cleanup_frme.sum()/ cleanup_frme['data_points_before_cleanup'].sum()

In [ ]:
cleanup_frme.head(2)

In [ ]:
cleanup_frme.to_csv(os.path.join(DIR_WRITE, f'data_cleanup.csv'))

writer = pd.ExcelWriter(os.path.join(DIR_WRITE, f'data_cleanup.xlsx'))
cleanup_frme.to_excel(writer, 'cleanup_frme')
writer.close()

### 7. Calculate ventilator parameters which are interpreted differently for SIPPV and SIMV 

#### A. Calculate the expired VT of ventilator inflations (VTe_kg) as appropriate for each ventilator mode

In [ ]:
for patient, dta in data_all.items():
    # Skip recordings which had only nCPAP
    if 'VTemand_kg' not in dta.columns:
        continue
    
    # For SIMV / SIMVPSV recordings take VTe_kg from VTemand_resp_kg, for SIPPV from VTemand_kg
    if 'SIMV' in dta['Ventilator_mode'].unique() or 'SIMVPSV' in dta['Ventilator_mode'].unique():
        #print(patient)
        dta['VTe_kg'] = np.where(dta['Ventilator_mode'].isin(['SIMV', 'SIMVPSV']), 
            dta['VTemand_resp_kg'], dta['VTemand_kg']) 
    else:
        dta['VTe_kg'] = dta['VTemand_kg']
    
    data_all[patient] = dta

#### B. Calculate the number of mandatory ventilator inflations 

In [ ]:
for patient, dta in data_all.items():
    # Skip recordings which had only nCPAP
    if 'VTemand_kg' not in dta.columns:
        continue
    
    # For SIMV / SIMVPSV recordings take RRmand from RR_set, for SIPPV from RR
    if 'SIMV' in dta['Ventilator_mode'].unique() or 'SIMVPSV' in dta['Ventilator_mode'].unique():
        if 'SIPPV' in dta['Ventilator_mode'].unique():
            dta['RRmand'] = np.where(dta['Ventilator_mode'].isin(['SIMV', 'SIMVPSV']), 
                dta['RR_set'], dta['RR']) 
        else:
            dta['RRmand'] = dta['RR_set']    
    else:
        dta['RRmand'] = dta['RR']
    
    data_all[patient] = dta

### 8. Descriptive statistics in the whole dataset

#### A. Statistics on individual cases

In [ ]:
percentiles = [0.05, 0.25, 0.5, 0.75, 0.95]
index_names = ['data_points', 'mean', 'SD', 'min', '5pc', '25pc', 'median', '75pc', '95pc', 'max']

stats_all_patients = {} 

for recording in data_all:
    stats_all_patients[recording] = round(data_all[recording].describe(percentiles = percentiles), 2)
    stats_all_patients[recording].index = index_names
    stats_all_patients[recording].dropna(how='all', subset=index_names[1:], axis=1, inplace=True)

In [ ]:
# Create table with statistics for all cases and all relevant parameters
stats_all_patients_combined = pd.concat(stats_all_patients, axis = 1).T

stats_all_patients_combined.info()

In [ ]:
stats_all_patients_combined

#### B. Statistics on selected individual parameters and setting

In [ ]:
# Selected individual parameters and settings

pars_selected = ['FiO2_set', 'MAP', 'PIP', 'PEEP', 'VTe_kg', 'RRmand', 'MV_kg', 'Ti_set', 'Te_set', 'Leak']

stats_all_parameters = {}

for par in pars_selected:
    stats_all_parameters[par] = stats_all_patients_combined.swaplevel(0,1).loc[par].sort_values('mean', ascending = False)

In [ ]:
# Unstack table to create table with different configuration
stats_all_parameters_combined = pd.concat(stats_all_parameters, axis=1)

stats_all_parameters_combined.head()

#### C. Group statistics on selected parameters

In [ ]:
pars_for_mean = ['PIP', 'MAP', 'PEEP', 'MV_kg', 'VTe_kg', 'RRmand',]

group_stats_all_mean = \
    round(stats_all_parameters_combined.swaplevel(axis=1)['mean'].describe()[pars_for_mean].T, 2)
group_stats_all_mean

In [ ]:
pars_for_median = ['FiO2_set', 'Leak', 'Ti_set', 'Te_set']

group_stats_all_median = \
    round(stats_all_parameters_combined.swaplevel(axis=1)['median'].describe()[pars_for_median].T, 2)
group_stats_all_median

#### C. Export statistics to a multisheet Excel file and pickle archive

In [ ]:
# Save statistics into Excel file
writer = pd.ExcelWriter(os.path.join(DIR_WRITE, 'stats_all_patients_ELBW.xlsx'))
for patient in stats_all_patients:
    stats_all_patients[patient].to_excel(writer, patient)
writer.close()

In [ ]:
# Save statistics into Excel file
writer = pd.ExcelWriter(os.path.join(DIR_WRITE, 'stats_all_parameters_ELBW.xlsx'))
for parameter in stats_all_parameters:
    stats_all_parameters[parameter].to_excel(writer, parameter)
writer.close()

In [ ]:
# Save statistics into Excel file
writer = pd.ExcelWriter(os.path.join(DIR_WRITE, 'stats_all_combined_ELBW.xlsx'))
stats_all_patients_combined.to_excel(writer, 'patients')
stats_all_parameters_combined.to_excel(writer, 'parameters')
writer.close()

In [ ]:
# Save group statistics into Excel file
writer = pd.ExcelWriter(os.path.join(DIR_WRITE, 'group_stats_all_ELBW.xlsx'))
group_stats_all_mean.to_excel(writer, 'parametric')
group_stats_all_median.to_excel(writer, 'non_parametric')
writer.close()

### 9. Combine data from all recordings, and export overall statistics about it as an Excel file

As this includes all data with various ventilator modes, some parameters should be interpreted with caution

In [ ]:
data_all_combined = pd.concat(data_all)

In [ ]:
data_all_combined.describe()

In [ ]:
writer = pd.ExcelWriter(os.path.join(DIR_WRITE, f'descriptive_stats_combined_ELBW.xlsx'))
cleanup_frme.to_excel(writer, 'stats_all_combined')
writer.close()

### 10. Analyse ventilator modes used

#### A. Ventilator modes

Some modes do not occur in this dataset

In [ ]:
vent_modes_ELBW.sum()

In [ ]:
# Limit vent mode table to modes which did occur
vent_modes_ELBW = vent_modes_ELBW[['SIMV', 'SIMVPSV', 'SIPPV', 'NCPAP', 'total', 'VG_on',]]
vent_modes_ELBW.head()

#### B. Assign recordings where >95% is one mode to that mode

In [ ]:
predominantly_sippv = set(vent_modes_ELBW[vent_modes_ELBW['SIPPV'] > vent_modes_ELBW['total'] * 0.95].index)
len(predominantly_sippv)

In [ ]:
vent_modes_ELBW.loc[sorted(predominantly_sippv)]

In [ ]:
predominantly_simv = set(vent_modes_ELBW[vent_modes_ELBW['SIMV'] > vent_modes_ELBW['total'] * 0.95].index)
len(predominantly_simv)

In [ ]:
vent_modes_ELBW.loc[sorted(predominantly_simv)]

In [ ]:
predominantly_simv_ps = set(vent_modes_ELBW[vent_modes_ELBW['SIMVPSV'] > vent_modes_ELBW['total'] * 0.95].index)
len(predominantly_simv_ps)

In [ ]:
vent_modes_ELBW.loc[sorted(predominantly_simv_ps)]

In [ ]:
# They are all fully nCPAP as these babies were not intubated
predominantly_ncpap = set(vent_modes_ELBW[vent_modes_ELBW['NCPAP'] > vent_modes_ELBW['total'] * 0.95].index)
len(predominantly_ncpap)

In [ ]:
vent_modes_ELBW.loc[sorted(predominantly_ncpap)]

In [ ]:
no_predominant_mode = set(vent_modes_ELBW.index) - (predominantly_simv | predominantly_simv_ps |
                          predominantly_sippv |  predominantly_ncpap)
vent_modes_ELBW.loc[sorted(no_predominant_mode)]

In [ ]:
both_simv_and_sippv = no_predominant_mode.copy()
# Exclude recordings which have SIMV and SIMVPSV
both_simv_and_sippv.discard('AL000152')
vent_modes_ELBW.loc[sorted(both_simv_and_sippv)]

In [ ]:
# Exclude recordings which have SIMV and SIPPV
both_simv_and_simvpsv = no_predominant_mode - both_simv_and_sippv
vent_modes_ELBW.loc[sorted(both_simv_and_simvpsv)]

#### Assign those ones with >95% VG or noVG to those modes

In [ ]:
predominantly_vg = set(vent_modes_ELBW[vent_modes_ELBW['VG_on'] > vent_modes_ELBW['total'] * 0.95].index)
vent_modes_ELBW.loc[sorted(predominantly_vg)]

In [ ]:
len(predominantly_vg)

In [ ]:
predominantly_novg = set(vent_modes_ELBW[vent_modes_ELBW['VG_on'] < vent_modes_ELBW['total'] * 0.05].index) & \
                     set(vent_modes_ELBW[vent_modes_ELBW['NCPAP'] < vent_modes_ELBW['total'] * 0.05].index)
vent_modes_ELBW.loc[sorted(predominantly_novg)]

In [ ]:
len(predominantly_novg)

In [ ]:
both_vg_novg = set(vent_modes_ELBW[ (vent_modes_ELBW['VG_on'] >= vent_modes_ELBW['total'] * 0.05) & 
                  (vent_modes_ELBW['VG_on'] <= vent_modes_ELBW['total'] * 0.95)].index)
vent_modes_ELBW.loc[sorted(both_vg_novg)]

#### C. Save tables with ventilator modes into Excel file

In [ ]:
writer = pd.ExcelWriter(os.path.join(DIR_WRITE, 'vent_modes_ELBW_categories.xlsx'))
vent_modes_ELBW.to_excel(writer, 'all')
vent_modes_ELBW.loc[sorted(predominantly_simv)].to_excel(writer, 'predominantly_simv')
vent_modes_ELBW.loc[sorted(predominantly_simv_ps)].to_excel(writer, 'predominantly_simv_ps')
vent_modes_ELBW.loc[sorted(predominantly_sippv)].to_excel(writer, 'predominantly_sippv')
vent_modes_ELBW.loc[sorted(predominantly_ncpap)].to_excel(writer, 'predominantly_ncpap')
vent_modes_ELBW.loc[sorted(no_predominant_mode)].to_excel(writer, 'no_predominant_mode')
vent_modes_ELBW.loc[sorted(both_simv_and_sippv)].to_excel(writer, 'both_simv_and_sippv')
vent_modes_ELBW.loc[sorted(both_simv_and_simvpsv)].to_excel(writer, 'both_simv_and_simvpsv')

vent_modes_ELBW.loc[sorted(predominantly_vg)].to_excel(writer, 'predominantly_vg')
vent_modes_ELBW.loc[sorted(predominantly_novg)].to_excel(writer, 'predominantly_novg')
vent_modes_ELBW.loc[sorted(both_vg_novg)].to_excel(writer, 'both_vg_novg')
writer.close()

### 11. Limit data to the given ventilator modes

In [ ]:
# Recordings with one mode only

# SIPPV only
data_sippv = {}
for case in predominantly_sippv:
    data_sippv[case] = data_all[case][data_all[case]['Ventilator_mode'] == 'SIPPV']

# SIMV only
data_simv = {}
for case in predominantly_simv:
    data_simv[case] = data_all[case][data_all[case]['Ventilator_mode'] == 'SIMV']

# NCPAP
# None of these recordings has any other mode as the baby was not intubated
data_ncpap = {}
for case in predominantly_ncpap:
    data_ncpap[case] = data_all[case][data_all[case]['Ventilator_mode']== 'NCPAP']

In [ ]:
# Recordings with more than one mode

# SIPPV and SIMV in the same recording
data_both_sippv_simv = {}
for case in both_simv_and_sippv:
    data_both_sippv_simv[case] = data_all[case][data_all[case]['Ventilator_mode'].isin(['SIMV', 'SIPPV'])]
    
# SIMV and SIMVPSV in the same recording
data_both_simv_simvpsv = {}
for case in both_simv_and_simvpsv:
    data_both_simv_simvpsv[case] = data_all[case][data_all[case]['Ventilator_mode'].isin(['SIMV', 'SIMVPSV'])]

In [ ]:
# VG only
data_vg = {}
for case in predominantly_vg:
    data_vg[case] = data_all[case][data_all[case]['VG_state'] == 'on']

# no VG only
# None of these recordings had any VG periods therefore (see above table), therefore there is nothing to remove
data_novg = {}
for case in predominantly_novg:
    data_novg[case] = data_all[case]

# both VG and no VG
data_both_vg_novg = {}
for case in both_vg_novg:
    data_both_vg_novg[case] = data_all[case]

### 12. Further process ventilator data specific to different ventilator modes

#### A. Recordings with SIPPV

In [ ]:
data_sippv_all = pd.concat(data_sippv)

In [ ]:
# For SIPPV MVresp should be n/a as it is always 100%
data_sippv_all = data_sippv_all.drop('MVresp', axis=1)

# During SIPPV VTemand_resp_kg and VTespon_resp_kg are not valid
data_sippv_all = data_sippv_all.drop(['VTemand_resp_kg', 'VTespon_pat_kg'], axis=1)

# Drop Flow_exp as no data
data_sippv_all = data_sippv_all.drop('Flow_exp', axis=1)

# RR_diff is the difference between the set and actual ventilator rate
data_sippv_all['RR_diff'] = data_sippv_all['RRmand'] - data_sippv_all['RR_set'] 

data_sippv_all.describe()

#### B. Recordings with SIMV 

In [ ]:
data_simv_all = pd.concat(data_simv)

In [ ]:
# Drop Flow_exp as no data
data_simv_all = data_simv_all.drop('Flow_exp', axis=1)

data_simv_all.describe()

#### C. Recordings with both SIPPV and SIMV

In [ ]:
data_both_sippv_simv_all = pd.concat(data_both_sippv_simv)

In [ ]:
# Drop Flow_exp as no data
data_both_sippv_simv_all = data_both_sippv_simv_all.drop('Flow_exp', axis=1)

data_both_sippv_simv_all.describe()

#### D. Recordings with both SIMV and SIMVPSV

In [ ]:
data_both_simv_simvpsv_all = pd.concat(data_both_simv_simvpsv)

data_both_simv_simvpsv_all.describe()

#### E. Recordings with nasal CPAP

In [ ]:
data_ncpap_all = pd.concat(data_ncpap)

In [ ]:
columns_to_keep = ['FiO2', 'Flow_exp', 'MAP', 'FiO2_set', 'PEEP_set', 'PIP_set', 'Ventilator_mode',]
data_ncpap_all = data_ncpap_all[columns_to_keep]

data_ncpap_all.describe()

#### F. Recordings with VG

For SIMV and SIMVPSV recordings `VTemand_resp_kg` is to be used as here there can be spontaneous breaths between ventilator inflation whose tidal volume is not targeted, for SIPPV use `VTemand_kg`

In [ ]:
for patient, dta in data_vg.items():
    
    # The difference between the actual and the target expired VT
    dta['VT_diff_kg'] = dta['VTe_kg'] - dta['VG_set_kg']
    # Calculate absolute value of the difference
    dta['VT_diff_kg_abs'] = np.abs(dta['VT_diff_kg'])

    dta['P_diff'] = dta['PIP_set'] - dta['PIP']
    # Calculate absolute value of the difference
    dta['P_diff_abs'] = np.abs(dta['P_diff'])
    
    data_vg[patient] = dta
    data_vg_all = pd.concat(data_vg)

# Drop Flow_exp as no data
data_vg_all = data_vg_all.drop('Flow_exp', axis=1)

data_vg_all.describe()

#### G. Recordings without VG

In [ ]:
for patient, dta in data_novg.items():
    
    data_novg[patient] = dta

data_novg_all = pd.concat(data_novg)

# Drop Flow_exp as no data and 'VGset_kg' as there is no target VT here
data_novg_all = data_novg_all.drop(['Flow_exp', 'VG_set_kg'], axis=1)

data_novg_all.describe()

#### H. Recordings with both VG and non-VG

In [ ]:
for patient, dta in data_both_vg_novg.items():
        
    # The difference between the actual and the target expired VT
    dta['VT_diff_kg'] = dta['VTe_kg'] - dta['VG_set_kg']
    # Calculate absolute value of the difference
    dta['VT_diff_kg_abs'] = np.abs(dta['VT_diff_kg'])

    dta['P_diff'] = dta['PIP_set'] - dta['PIP']
    # Calculate absolute value of the difference
    dta['P_diff_abs'] = np.abs(dta['P_diff'])
    
    data_both_vg_novg[patient] = dta
    
    data_both_vg_novg[patient] = dta
    data_both_vg_novg_all = pd.concat(data_both_vg_novg)

# Drop Flow_exp as no data and 'VGset_kg' as there is no target VT here
# Also, there is no SIMV, so remove SIMV specific parameters
columns_to_drop = ['Flow_exp', 'MVresp', 'VTemand_resp_kg','VTespon_pat_kg']
data_both_vg_novg_all = data_both_vg_novg_all.drop(columns_to_drop, axis=1)

data_both_vg_novg_all.describe()

#### I. Save descriptive statistics about the subsets with different ventilator modes in Excel file

In [ ]:
# Save statistics into Excel file
writer = pd.ExcelWriter(os.path.join(DIR_WRITE, 'descriptive_stats_vent_modes_ELBW.xlsx'))
data_sippv_all.describe().to_excel(writer, 'sippv')
data_simv_all.describe().to_excel(writer, 'simv')
data_both_sippv_simv_all.describe().to_excel(writer, 'both_sippv_simv')
data_both_simv_simvpsv_all.describe().to_excel(writer, 'both_simv_simvpsv')

data_ncpap_all.describe().to_excel(writer, 'ncpap')

data_vg_all.describe().to_excel(writer, 'vg')
data_novg_all.describe().to_excel(writer, 'novg')
data_both_vg_novg_all.describe().to_excel(writer, 'both_vg_novg')

writer.close()

### 13. Compare SIPPV and SIMV in different recordings

In [ ]:
clin_data_comput_ELBW.loc[sorted(predominantly_sippv)].describe()

In [ ]:
clin_data_comput_ELBW.loc[sorted(predominantly_simv)].describe()

In [ ]:
for par in ['Gestational Age (weeks)_comput', 'Birth Weight_comput']:
    group_1 = clin_data_comput_ELBW.loc[sorted(predominantly_sippv)][par]
    group_2 = clin_data_comput_ELBW.loc[sorted(predominantly_simv)][par]
    print(par, '\n', stats.mannwhitneyu(group_1, group_2,), '\n')

In [ ]:
# Ventilator parameters with normal distribution
pars_for_mean = ['PIP', 'MAP', 'PEEP', 'MV_kg', 'VTe_kg', 'VTimand_kg', 'VTemand_kg', 'RRmand', 'RR_set', ]
# Ventilator parameters which are not normally distributed
pars_for_median = ['FiO2_set', 'Leak', 'RR_diff',]

sippv_stats_mean = round(data_sippv_all.groupby(level=0)[pars_for_mean].describe().swaplevel(axis=1)['mean'].describe().T, 1)
sippv_stats_median = round(data_sippv_all.groupby(level=0)[pars_for_median].describe().swaplevel(axis=1)['50%'].describe().T, 1)

In [ ]:
sippv_stats_mean

In [ ]:
sippv_stats_median

In [ ]:
# Ventilator parameters with normal distribution
pars_for_mean = ['PIP', 'MAP', 'PEEP', 'MV_kg', 'MVresp', 'VTe_kg', 'VTimand_kg', 'VTemand_kg', 
                 'VTemand_resp_kg', 'VTespon_pat_kg', 'RRmand', 'RR_set']
# Ventilator parameters which are not normally distributed
pars_for_median = ['FiO2_set', 'Leak',]

simv_stats_mean = round(data_simv_all.groupby(level=0)[pars_for_mean].describe().swaplevel(axis=1)['mean'].describe().T, 2)
simv_stats_median = round(data_simv_all.groupby(level=0)[pars_for_median].describe().swaplevel(axis=1)['50%'].describe().T, 2)

In [ ]:
simv_stats_mean

In [ ]:
simv_stats_median

In [ ]:
pars = ['PIP', 'MAP', 'PEEP', 'MV_kg', 'VTe_kg', 'RRmand', 'RR_set', 'FiO2_set', 'Leak', ]
# The aggregated values (means, medians) themselves do not show normal distribution in the groups, 
# use nonparametric measures and tests
stts = ['count', 'min', '25%', '50%', '75%', 'max']

sippv_stats = pd.concat([sippv_stats_mean[stts], sippv_stats_median[stts]])
simv_stats = pd.concat([simv_stats_mean[stts], simv_stats_median[stts]])
sippv_simv_stats = pd.merge(sippv_stats, simv_stats, left_index=True, right_index=True, 
                            suffixes= ['_sippv', '_simv']).sort_index(axis=1, ascending=False)
sippv_simv_stats = sippv_simv_stats.loc[pars]
sippv_simv_stats

##### Inferential statistics for selected parameters, SIPPV vs SIMV

In [ ]:
# Mann-Whitney test

for par in ['PIP', 'MAP', 'PEEP', 'MV_kg', 'VTe_kg', 'RRmand', 'RR_set']:
    # Calculate mean of these parameters in each recording and use them in group comparison
    group_1 = data_sippv_all.groupby(level=0)[par].mean()
    group_2 = data_simv_all.groupby(level=0)[par].mean()
    # The aggregated values (means, medians) themselves do not show normal distribution in the groups, 
    # use nonparametric measures and tests
    print(par, '\n', stats.mannwhitneyu(group_1.dropna(), group_2.dropna(),), '\n')

In [ ]:
# Mann-Whitney test

for par in ['FiO2_set', 'Leak',]:
    # Calculate median of these parameters in each recording and use them in group comparison
    group_1 = data_sippv_all.groupby(level=0)[par].median()
    group_2 = data_simv_all.groupby(level=0)[par].median()
    # The aggregated values (means, medians) themselves do not show normal distribution in the groups, 
    # use nonparametric measures and tests
    print(par, '\n', stats.mannwhitneyu(group_1.dropna(), group_2.dropna(),), '\n')

### 14. Compare SIMV and SIPPV in those recordings when both occurred

In [ ]:
data_both_sippv_simv_all = pd.concat(data_both_sippv_simv)
data_both_sippv_simv_all.index.set_names(['recording', 'datetime'], inplace=True)
data_both_sippv_simv_all = data_both_sippv_simv_all.reset_index().set_index(['recording', 'Ventilator_mode'])

In [ ]:
data_both_sippv_simv_all.groupby(level=[0,1], observed=False).size()

- AL000089: SIMV-VG -> SIPPV-VG
- AL000104: SIPPV-VG -> SIMV-VG
- AL000246: SIMV-VG -> SIPPV-VG
- AL000465: SIPPV-VG -> SIMV-VG
- AL000687: SIMV-VG -> SIPPV-VG
- AT000009: SIPPV-VG -> SIMV-VG
- AT000578: SIPPV-VG -> SIMV-VG

In [ ]:
stats_sippv_simv_mean = data_both_sippv_simv_all.groupby(level=[0,1],
            observed=False)[['PIP', 'PEEP', 'MAP', 'MV_kg', 'MVresp', 'RRmand', 'VTemand_kg', 'VTemand_resp_kg', 'VTespon_pat_kg']].mean().unstack()
stats_sippv_simv_mean

In [ ]:
stats_sippv_simv_mean.describe()

In [ ]:
stats.ttest_rel(stats_sippv_simv_mean['PIP']['SIPPV'], stats_sippv_simv_mean['PIP']['SIMV'])

In [ ]:
stats.ttest_rel(stats_sippv_simv_mean['PEEP']['SIPPV'], stats_sippv_simv_mean['PEEP']['SIMV'])

In [ ]:
stats.ttest_rel(stats_sippv_simv_mean['MAP']['SIPPV'], stats_sippv_simv_mean['MAP']['SIMV'])

In [ ]:
stats.ttest_rel(stats_sippv_simv_mean['MV_kg']['SIPPV'], stats_sippv_simv['MV_kg']['SIMV'])

In [ ]:
stats.ttest_rel(stats_sippv_simv_mean['RRmand']['SIPPV'], stats_sippv_simv['RRmand']['SIMV'])

In [ ]:
stats.ttest_rel(stats_sippv_simv_mean['VTemand_kg']['SIPPV'], stats_sippv_simv['VTemand_kg']['SIMV'])

In [ ]:
stats_sippv_simv_median = data_both_sippv_simv_all.groupby(level=[0,1],
            observed=False)[['FiO2_set', 'Leak']].median().unstack()
stats_sippv_simv_median

In [ ]:
stats_sippv_simv_median.describe()

In [ ]:
stats.ttest_rel(stats_sippv_simv_median['FiO2_set']['SIPPV'], stats_sippv_simv_median['FiO2_set']['SIMV'])

In [ ]:
stats.ttest_rel(stats_sippv_simv_median['Leak']['SIPPV'], stats_sippv_simv_median['Leak']['SIMV'])

### 15. Compare VG and no VG recordings



In [ ]:
clin_data_comput_ELBW.loc[sorted(predominantly_vg)].describe()

In [ ]:
clin_data_comput_ELBW.loc[sorted(predominantly_novg)].describe()

In [ ]:
for par in ['Gestational Age (weeks)_comput', 'Birth Weight_comput']:
    group_1 = clin_data_comput_ELBW.loc[sorted(predominantly_vg)][par]
    group_2 = clin_data_comput_ELBW.loc[sorted(predominantly_novg)][par]
    print(par, '\n', stats.mannwhitneyu(group_1, group_2,), '\n')

In [ ]:
# Ventilator parameters with normal distribution
pars_for_mean = ['PIP', 'MAP', 'PEEP', 'MV_kg', 'VTe_kg', 'VG_set_kg',  'VTimand_kg', 'VTemand_kg', 'RRmand', 'RR_set']
# Ventilator parameters which are not normally distributed
pars_for_median = ['FiO2_set', 'Leak', 'VT_diff_kg', 'VT_diff_kg_abs', 'P_diff', 'P_diff_abs']

vg_stats_mean = round(data_vg_all.groupby(level=0)[pars_for_mean].describe().swaplevel(axis=1)['mean'].describe().T, 2)
vg_stats_median = round(data_vg_all.groupby(level=0)[pars_for_median].describe().swaplevel(axis=1)['50%'].describe().T, 2)

In [ ]:
vg_stats_mean

In [ ]:
vg_stats_median

In [ ]:
# Ventilator parameters with normal distribution
pars_for_mean = ['PIP', 'MAP', 'PEEP', 'MV_kg', 'VTe_kg', 'VTimand_kg', 'VTemand_kg', 'RRmand', 'RR_set']
# Ventilator parameters which are not normally distributed
pars_for_median = ['FiO2_set', 'Leak', ]

novg_stats_mean = round(data_novg_all.groupby(level=0)[pars_for_mean].describe().swaplevel(axis=1)['mean'].describe().T, 2)
novg_stats_median = round(data_novg_all.groupby(level=0)[pars_for_median].describe().swaplevel(axis=1)['50%'].describe().T, 2)

In [ ]:
novg_stats_mean

In [ ]:
novg_stats_median

In [ ]:
pars = ['PIP', 'MAP', 'PEEP', 'MV_kg', 'VTe_kg', 'RRmand', 'RR_set', 'FiO2_set', 'Leak', ]
# The aggregated values (means, medians) themselves do not show normal distribution in the groups, 
# use nonparametric measures and tests
stts = ['count', 'min', '25%', '50%', '75%', 'max']

vg_stats = pd.concat([vg_stats_mean[stts], vg_stats_median[stts]])
novg_stats = pd.concat([novg_stats_mean[stts], novg_stats_median[stts]])
vg_novg_stats = pd.merge(vg_stats,  novg_stats, left_index=True, right_index=True, 
                            suffixes= ['_vg', '_novg']).sort_index(axis=1, ascending=False)
vg_novg_stats = vg_novg_stats.loc[pars]
vg_novg_stats

##### Inferential statistics for selected parameters, VG vs noVG

In [ ]:
# Mann-Whitney test

for par in ['PIP', 'MAP', 'PEEP', 'MV_kg', 'VTe_kg', 'RRmand', 'RR_set']:
    # Calculate mean of these parameters in each recording and use them in group comparison
    group_1 = data_vg_all.groupby(level=0)[par].mean()
    group_2 = data_novg_all.groupby(level=0)[par].mean()
    # The aggregated values (means, medians) themselves do not show normal distribution in the groups, 
    # use nonparametric measures and tests
    print(par, '\n', stats.mannwhitneyu(group_1.dropna(), group_2.dropna(),), '\n')

In [ ]:
# Mann-Whitney test

for par in ['FiO2_set', 'Leak',]:
    # Calculate median of these parameters in each recording and use them in group comparison
    group_1 = data_vg_all.groupby(level=0)[par].median()
    group_2 = data_novg_all.groupby(level=0)[par].median()
    # The aggregated values (means, medians) themselves do not show normal distribution in the groups, 
    # use nonparametric measures and tests
    print(par, '\n', stats.mannwhitneyu(group_1.dropna(), group_2.dropna(),), '\n')

### 16. Compare VG and noVG in those recordings when both occurred

- VTemand, PIP and MV and FiO2 in VG and noVG recordings
- VTdiff and Pdiff in VG recordings

In [ ]:
data_both_vg_novg_all.index.set_names(['recording', 'datetime'], inplace=True)
data_both_vg_novg_all = data_both_vg_novg_all.reset_index().set_index(['recording', 'VG_state'])

In [ ]:
data_both_vg_novg_all.groupby(level=[0,1], observed=False).size()

In [ ]:
data_both_vg_novg_all.groupby(level=[0,1], observed=False)[['MV_kg','RRmand', 'VTe_kg']].mean().unstack()

In [ ]:
data_both_vg_novg_all.groupby(level=[0,1], observed=False)[['MV_kg','RRmand', 'VTe_kg']].mean().unstack().mean()

- AL000110: 
- AL000115: 
- AL001019: 
- AT000437: 
- AT000613: 

In [ ]:
stats_vg_novg_mean = data_both_vg_novg_all.groupby(level=[0,1],
            observed=False)[['MAP', 'MV_kg', 'PEEP', 'PIP', 'RR',
       'VTemand_kg', 'VTimand_kg', 'VG_set_kg', 'VTe_kg', 'RRmand',]].mean().unstack()
stats_vg_novg_mean

In [ ]:
stats_vg_novg_mean.describe()

In [ ]:
stats.wilcoxon(stats_vg_novg_mean['MAP']['off'], stats_vg_novg_mean['MAP']['on'])

In [ ]:
stats.wilcoxon(stats_vg_novg_mean['PIP']['off'], stats_vg_novg_mean['PIP']['on'])

In [ ]:
stats.wilcoxon(stats_vg_novg_mean['PEEP']['off'], stats_vg_novg_mean['PEEP']['on'])

In [ ]:
stats.wilcoxon(stats_vg_novg_mean['MV_kg']['off'], stats_vg_novg_mean['MV_kg']['on'])

In [ ]:
stats.wilcoxon(stats_vg_novg_mean['RR']['off'], stats_vg_novg_mean['RR']['on'])

In [ ]:
stats.wilcoxon(stats_vg_novg_mean['VTe_kg']['off'], stats_vg_novg_mean['VTe_kg']['on'])

In [ ]:
stats.wilcoxon(stats_vg_novg_mean['VTemand_kg']['off'], stats_vg_novg_mean['VTemand_kg']['on'])

In [ ]:
stats.wilcoxon(stats_vg_novg_mean['VTimand_kg']['off'], stats_vg_novg_mean['VTimand_kg']['on'])

In [ ]:
stats_vg_novg_median  = data_both_vg_novg_all.groupby(level=[0,1],
            observed=False)[['Leak', 'FiO2_set', ]].median().unstack()
stats_vg_novg_median

In [ ]:
stats.wilcoxon(stats_vg_novg_median['Leak']['off'], stats_vg_novg_median['Leak']['on'])

### 17. Analyse blood gases

-  Blood gases at the arrival to NICU for all cases and separately for SIPPV, SIMV and VG, non-VG
-  pCO2s at the end of transfer vs VTemand and MV

#### A. Blood gases at the end of transfer

Blood gases were available in 47/55 and 50/50 cases, during arrival and at handover, respectively.
None of them was arterial

In [ ]:
# All cases, arrival of Cerny
blood_gases_arrival = blood_gases_manual['At arrival of PCAM'][['pH', 'pCO2', 'BE']]
blood_gases_arrival.describe()

In [ ]:
# All cases, handover on NICU
blood_gases_handover = blood_gases_manual['At arrival to NICU'][['pH', 'pCO2', 'BE']]
blood_gases_handover.describe()

In [ ]:
# SIPPV, arrival of Cerny
recs_available_sippv = set(blood_gases_arrival.index) & predominantly_sippv
blood_gases_arrival.loc[sorted(recs_available_sippv)].describe()

In [ ]:
# SIPPV, handover on NICU
recs_available_sippv = set(blood_gases_handover.index) & predominantly_sippv
blood_gases_handover.loc[sorted(recs_available_sippv)].describe()

In [ ]:
# SIMV, arrival of Cerny
recs_available_simv = set(blood_gases_arrival.index) & predominantly_simv
blood_gases_arrival.loc[sorted(recs_available_simv)].describe()

In [ ]:
# SIMV, handover on NICU
recs_available_simv = set(blood_gases_handover.index) & predominantly_simv
blood_gases_handover.loc[sorted(recs_available_simv)].describe()

In [ ]:
# Mann-Whitney test, blood gases at arrival of Cerny
for par in ['pH', 'pCO2', 'BE']:
    group_1 = blood_gases_arrival.loc[sorted(recs_available_sippv)][par]
    group_2 = blood_gases_arrival.loc[sorted(recs_available_simv)][par]
    print(par, '\n', stats.mannwhitneyu(group_1.dropna(), group_2.dropna(),), '\n')

In [ ]:
# Mann-Whitney test, blood gases on handover to NICU
for par in ['pH', 'pCO2', 'BE']:
    group_1 = blood_gases_handover.loc[sorted(recs_available_sippv)][par]
    group_2 = blood_gases_handover.loc[sorted(recs_available_simv)][par]
    print(par, '\n', stats.mannwhitneyu(group_1.dropna(), group_2.dropna(),), '\n')

In [ ]:
# VG, arrival of Cerny
recs_available_vg = set(blood_gases_arrival.index) & predominantly_vg
blood_gases_arrival.loc[sorted(recs_available_vg)].describe()

In [ ]:
# VG, handover on NICU
recs_available_vg = set(blood_gases_handover.index) & predominantly_vg
blood_gases_handover.loc[sorted(recs_available_vg)].describe()

In [ ]:
# No VG, arrival of Cerny
recs_available_novg = set(blood_gases_arrival.index) & predominantly_novg
blood_gases_arrival.loc[sorted(recs_available_novg)].describe()

In [ ]:
# No VG, handover on NICU
recs_available_novg = set(blood_gases_handover.index) & predominantly_novg
blood_gases_handover.loc[sorted(recs_available_novg)].describe()

In [ ]:
# Mann-Whitney test, blood gases at arrival of Cerny
for par in ['pH', 'pCO2', 'BE']:
    group_1 = blood_gases_arrival.loc[sorted(recs_available_vg)][par]
    group_2 = blood_gases_arrival.loc[sorted(recs_available_novg)][par]
    print(par, '\n', stats.mannwhitneyu(group_1.dropna(), group_2.dropna(),), '\n')

In [ ]:
# Mann-Whitney test, blood gases at handover on NICU
for par in ['pH', 'pCO2', 'BE']:
    group_1 = blood_gases_handover.loc[sorted(recs_available_vg)][par]
    group_2 = blood_gases_handover.loc[sorted(recs_available_novg)][par]
    print(par, '\n', stats.mannwhitneyu(group_1.dropna(), group_2.dropna(),), '\n')

#### B. Relationship between pCO2 at the end of transfer and VTe or MV

For this, use computationally collected gases

In [ ]:
blood_gases_frme = pd.concat(blood_gases_comput)
blood_gases_frme.index.names = ['case', 'datetime']
blood_gases_frme.reset_index(inplace=True)

In [ ]:
blood_gases_last_frme = blood_gases_frme.drop_duplicates(subset='case', keep='last')
blood_gases_last_frme.set_index('case', inplace=True)
blood_gases_last_frme.head(2)

In [ ]:
SIPPV_MV_CO2 = pd.merge(left = data_sippv_all.groupby(level=0)[['MV_kg']].mean(),
    right = blood_gases_last_frme['pCO2'].astype('float'), how='inner', left_index=True, right_index=True).dropna(how='any')
SIPPV_MV_CO2.head(2)

In [ ]:
SIMV_MV_CO2 = pd.merge(left = data_simv_all.groupby(level=0)[['MV_kg']].mean(),
    right = blood_gases_last_frme['pCO2'].astype('float'), how='inner', left_index=True, right_index=True).dropna(how='any')

SIMV_MV_CO2.head(2)

In [ ]:
SIPPV_SIMV_MV_CO2 = pd.concat([SIPPV_MV_CO2, SIMV_MV_CO2], keys = ['SIPPV', 'SIMV'])

In [ ]:
dpi=300; filetype = 'jpg'

fig,ax = plt.subplots(1,1, figsize=(6,6))
ax.scatter(SIPPV_MV_CO2['MV_kg'], SIPPV_MV_CO2['pCO2'], marker='o', facecolors='none', color ='black')
ax.scatter(SIMV_MV_CO2['MV_kg'], SIMV_MV_CO2['pCO2'], marker = 'x', color = 'black')
ax.set_ylim(0,100)
ax.set_xlabel('MV [L/min/kg]')
ax.set_ylabel('pCO$_2$ [mmHg]')
fig.savefig(os.path.join(DIR_WRITE, f'MV_CO2.{filetype}'), dpi = dpi, format = filetype, bbox_inches='tight', pad_inches=0.1);

In [ ]:
VG_VT_CO2 = pd.merge(left = data_vg_all.groupby(level=0)[['VTemand_kg']].mean(),
    right = blood_gases_last_frme['pCO2'].astype('float'), how='inner', left_index=True, right_index=True).dropna(how='any')
VG_VT_CO2.head(2)

In [ ]:
noVG_VT_CO2 = pd.merge(left = data_novg_all.groupby(level=0)[['VTemand_kg']].mean(),
    right = blood_gases_last_frme['pCO2'].astype('float'), how='inner', left_index=True, right_index=True).dropna(how='any')
noVG_VT_CO2.head(2)

In [ ]:
VG_noVG_VTCO2 = pd.concat([VG_VT_CO2, noVG_VT_CO2], keys = ['VG', 'noVG'])

In [ ]:
dpi=300; filetype = 'jpg'

fig,ax = plt.subplots(1,1, figsize=(6,6))
ax.scatter(VG_VT_CO2['VTemand_kg'], VG_VT_CO2['pCO2'], marker='o', facecolors='none', color ='black')
ax.scatter(noVG_VT_CO2['VTemand_kg'], noVG_VT_CO2['pCO2'], marker = 'x', color = 'black')
ax.hlines(37.5, 0, 10, color='black', linestyle='dashed')
ax.hlines(60, 0, 10, color='black', linestyle='dashed')
ax.vlines(4, 0, 100, color='black', linestyle='dashed')
ax.vlines(6, 0, 100, color='black', linestyle='dashed')
ax.set_xlim(0, 10)
ax.set_ylim(0,100)
ax.set_xlabel('VTe [mL/kg]')
ax.set_ylabel('pCO$_2$ [mmHg]')
fig.savefig(os.path.join(DIR_WRITE, f'VT_CO2.{filetype}'), dpi = dpi, format = filetype, bbox_inches='tight', pad_inches=0.1);